## MODEL TESTING

## Libraries, Constants, Functions

Importing libraries, constants, and functions from separate .py files, to enhance efficiency and promote code reuse across multiple notebooks.

To limit the number of calculations, the number of closest AEDs considered is reduced from 10 to 8.

In [ ]:
from libraries import *
from constants import *
from functions import *

CLOSEST_AEDS = 8

## Preparing data for calculations

Loading AED data, interventions data, potential locations for AEDs and indicators for optimal locations for each city and storing them in dictionaries. For interventions, this time the test data is loaded, as the objective now is to evaluate how well the optimal locations (calculated on the training data) cover the new unseen dataset of cardiac arrests.

In [ ]:
aeds = {}
cards = {}
possible_locations = {}
optimal_indicators = {}

for city in cities:
    os.chdir(data_path + clean_path)
    aeds[city] = pd.read_csv(city + "_aeds.csv")
    cards[city] = pd.read_csv(city + "_cards_test.csv")

    os.chdir(data_path + possible_locations_path)
    possible_locations[city] = pd.read_csv(city + "_possible_locations.csv")

    os.chdir(data_path + optimal_indicators_path)
    optimal_indicators[city] = pd.read_csv(city + "_optimal_indicators.csv")

Retaining only latitude and longitude locations, as for calculating cost matrices only those are necessary. Now, only optimal locations are kept.

In [ ]:
optimal_locations = {}

for city in cities:
    aeds[city] = aeds[city][['latitude', 'longitude']]
    possible_locations[city][['latitude', 'longitude']] = possible_locations[city]['geometry'].apply(
        lambda x: pd.Series(get_coordinates_from_geometry(x))
    )
    possible_locations[city] = possible_locations[city][['latitude', 'longitude']]
    
    combined_locations = pd.concat([aeds[city], possible_locations[city]], ignore_index=True)
    flag = optimal_indicators[city]["SelectionStatus"]
    optimal_locations[city] = combined_locations[flag == True]
    
    cards[city] = cards[city][['latitude', 'longitude']]

## Calculating cost matrices

Cost matrices are calculated separately for the old AED placements and for the new (optimal) AED placements, in order to compare them

In [ ]:
cost_matrices_old = {}
cost_matrices_new = {}

for city in cities:
    print("Current city: " + city)
    
    cost_matrices_old[city] = get_cost_matrix(cards, aeds, city, CLOSEST_AEDS)
    cost_matrices_new[city] = get_cost_matrix(cards, optimal_locations, city, CLOSEST_AEDS)

## Evaluation of coverage

Both old and new (optimal) coverage percentages are calculated, and the results are exported to a CSV file.

In [ ]:
os.chdir(data_path + app_path)
coverage_data = []

for city in cities:
    old_aeds_cost_matrix = cost_matrices_old[city]
    new_aeds_cost_matrix = cost_matrices_new[city]

    # Calculate if there's an AED within 150m for old and new placements
    old_aed_near_card = old_aeds_cost_matrix.apply(lambda row: (row < COVERAGE_RADIUS).any(), axis=1)
    new_aed_near_card = new_aeds_cost_matrix.apply(lambda row: (row < COVERAGE_RADIUS).any(), axis=1)
    
    coverage_data.append({
        'city': city,
        'old_coverage': old_aed_near_card.mean() * 100,
        'new_coverage': new_aed_near_card.mean() * 100
    })

pd.DataFrame(coverage_data).to_csv("coverage.csv", index = False)

## Creating datasets for dashboard

Exporting all the previous data for each city for the app to utilize. This includes information on each cardiac arrest and its closest AED in the new (optimal) placements. Additionally, information on both old and new AED locations is exported.

In [ ]:
os.chdir(data_path + clean_path)
all_dataframes = []

for city in cities:
    # Taking column names (which are coordinates), not the full ost matrices
    old_aed_columns = cost_matrices_old[city].columns
    old_aeds_locations = pd.DataFrame(old_aed_columns, columns=["Old AEDs"])
    old_aeds_locations[['lat', 'lon']] = old_aeds_locations['Old AEDs'].apply(make_coordinates_tuple).apply(pd.Series)
    old_aeds_locations.drop(columns=['Old AEDs'], inplace=True)
    
    new_aed_columns = cost_matrices_new[city].columns
    new_aeds_locations = pd.DataFrame(new_aed_columns, columns=["New AEDs"])
    new_aeds_locations[['lat', 'lon']] = new_aeds_locations['New AEDs'].apply(make_coordinates_tuple).apply(pd.Series)
    new_aeds_locations.drop(columns=['New AEDs'], inplace=True)

    # Importing cards
    cards = pd.read_csv(city + "_cards_test.csv")[['latitude', 'longitude']]
    cards.rename(columns={'latitude': 'lat'}, inplace=True)
    cards.rename(columns={'longitude': 'lon'}, inplace=True)

    new_aeds_cost_matrix = cost_matrices_new[city].apply(pd.to_numeric, errors='coerce')
    min_indices = np.argmin(new_aeds_cost_matrix.to_numpy(), axis=1)

    cards = pd.DataFrame({
        'lat': cards['lat'].values,
        'lon': cards['lon'].values,
        'aed_lat': new_aeds_locations.iloc[min_indices]['lat'].values,
        'aed_lon': new_aeds_locations.iloc[min_indices]['lon'].values
    })
    cards['distance'] = None
    for i, which_min_index in enumerate(min_indices):
        cards.at[i, 'distance'] = new_aeds_cost_matrix.iat[i, which_min_index]

    # Add type and city columns
    cards['type'] = 'card'
    old_aeds_locations['type'] = 'old_aed'
    new_aeds_locations['type'] = 'new_aed'
    cards['city'] = city
    old_aeds_locations['city'] = city
    new_aeds_locations['city'] = city

    # Add missing columns to old_aeds_locations and new_aeds_locations
    old_aeds_locations['aed_lat'] = np.nan
    old_aeds_locations['aed_lon'] = np.nan
    old_aeds_locations['distance'] = np.nan
    new_aeds_locations['aed_lat'] = np.nan
    new_aeds_locations['aed_lon'] = np.nan
    new_aeds_locations['distance'] = np.nan

    # Reorder columns to match the cards dataframe
    old_aeds_locations = old_aeds_locations[['lat', 'lon', 'aed_lat', 'aed_lon', 'distance', 'type', 'city']]
    new_aeds_locations = new_aeds_locations[['lat', 'lon', 'aed_lat', 'aed_lon', 'distance', 'type', 'city']]
    cards = cards[['lat', 'lon', 'aed_lat', 'aed_lon', 'distance', 'type', 'city']]

    # Concatenate dataframes for the current city
    all_dataframes.append(pd.concat([cards, old_aeds_locations, new_aeds_locations], ignore_index=True))

os.chdir(data_path + app_path)
app_data = pd.concat(all_dataframes, ignore_index = True)
app_data.to_csv("app_data.csv", index = False)

Exporting cards dataframe which contains infromation for the density maps on the dashboard

In [ ]:
cards_df = app_data[app_data['type'] == 'card']

longitude = cards_df['lon'].values
latitude = cards_df['lat'].values
# Combine longitude and latitude into a single array for KDE estimation
coordinates = np.vstack([longitude, latitude])

# Calculate density using KDE
kde = gaussian_kde(coordinates)
density = kde(coordinates)
cards_df['density'] = density

# Now cards_df contains the density values calculated based on longitude and latitude for type "Cards"
cards_df.to_csv("cards_with_density.csv",index=False)